In [3]:
import pandas as pd
import numpy as np
import pymysql
import datetime as dt
import yfinance as yf


pd.set_option("display.max_rows", 200)

In [4]:
#### TIMING THE SCRIPT FOR RUNNING:
run_start = dt.datetime.now()
run_start

datetime.datetime(2024, 1, 19, 13, 34, 23, 76722)

In [5]:
### calendario anbima:

file = '/home/peon/Documents/feriados_nacionais.xls'
df_holly = pd.read_excel(file)
df_holly = df_holly[~df_holly.Feriado.isnull()]

#### SQL QUERIES

In [6]:
import mysql.connector

cnx = mysql.connector.connect(user = 'peon',
                       password = 'Gotadelimao123',
                       host = 'localhost',
                       database = 'stocks')

cursor = cnx.cursor()

def sql_df(algo, columns):
    
    
    cursor.execute(algo)
    df = pd.DataFrame(cursor.fetchall())
    df.columns = columns
    df = df.sort_values('Date', ascending= False)
    
    
    return df

In [7]:
df_x = sql_df("SELECT * FROM extrato_b3", ['id', 'IN_OUT', 'Date', 'Evento', 
                                           'Ticker', 'Custodia', 'Quantidade', 'PU', 'Valor']
          )

df_y = sql_df("SELECT * FROM Carteira", 
           ['Date' , 'Ticker', 'Qt', 'PU', 'ValorAplicado', 'PUMercado', 'ValorMercado', 'Return']
          )


In [8]:
df_y.head()

,Date,Ticker,Qt,PU,ValorAplicado,PUMercado,ValorMercado,Return
7275,2024-01-18,GOAU4,47,11.52,541.55,9.70,455.90,-15.82
7267,2024-01-18,RANI3,74,11.09,820.37,9.95,736.30,-10.25
7259,2024-01-18,CMIN3,166,4.11,683.00,6.68,1108.88,62.53
7260,2024-01-18,CBAV3,101,4.20,424.23,4.12,416.12,-1.90
7261,2024-01-18,TAEE11,31,35.14,1089.29,37.22,1153.82,5.92


#### HERE WE FILTER OLDER TRANSACTIONS;

In [2]:
## removes transactions that are older than the last updated date of the wallet;
df = df_x.copy()

#date_last_tsc_record = df_x.Date.sort_values().max() #a data mais recente de transação;

if not df_y.empty:
    
    date_last_updated_wlt = df_y.Date.sort_values().max() # ultimo dia de update da carteira;
    df = df[df.Date > date_last_updated_wlt]

    # if we already have a last updated wallet, lets start our dictionary with that.
    last_pos = df_y[df_y.Date == date_last_updated_wlt][
            ['Qt', 'PU', 'ValorAplicado', 'Ticker']
        ].set_index('Ticker')

    # the query from sql returns a nonfloat type.
    last_pos = last_pos.astype(float)
    last_pos.Qt = last_pos.Qt.astype(int)
    
else:
    last_pos = pd.DateFrame() #empty dataframe for compatibility
    date_last_updated_wlt = None

NameError: name 'df_x' is not defined

In [1]:
date_last_updated_wlt

NameError: name 'date_last_updated_wlt' is not defined

In [10]:
### stop the script if no new transactions happened.
if df.empty:
    print('fail, so exit.')
    exit()

fail, so exit.


#### Topic

In [11]:
### PRE START

df = df.drop(columns = ['id'])

### remap first, to be easily queryable.;;
new_vals = {'Transferência - Liquidação':'liquidacao', 
            'Juros Sobre Capital Próprio':'jcp', 
            'Rendimento':'rend', 
            'Dividendo':'div', 
            'Transferência':'transferencia', 
            'Leilão de Fração':'leilao_de_fracao', 
            'Fração em Ativos':'fracao_ativos', 
            'Bonificação em Ativos':'bonificacao',
            'Direito de Subscrição': 'subsc'}


# replace,
df = df.replace({'Evento': new_vals})
df.rename(columns = {'Valor': 'ValorAplicado'}, inplace = True)

# DateTypes

df['Quantidade'] = df.Quantidade.astype(int)
df[['PU', 'ValorAplicado']] = df.loc[:, ['PU', 'ValorAplicado']].astype(float).apply(lambda x: round(x,2))
df = df.sort_values('Date')
#-------
df.tail(3)

,IN_OUT,Date,Evento,Ticker,Custodia,Quantidade,PU,ValorAplicado


### SUB DFs

In [12]:
# dataframe dos proventos

df_proventos = df[(df.Evento =='jcp') |
                  (df.Evento == 'div')].drop(columns = ['Custodia', 'Evento']).sort_index()


# dataframe das transacoes

df_trade = df[(df.Evento == 'liquidacao')].drop(columns = ['Custodia', 'Evento'])

## precisa remover os ticks de subscrição, pois não há compra.

mask_subsc = [True if x in df[df.Evento == 'subsc'].Ticker.values else False for x in df_trade.Ticker.values]
df_trade = df_trade[~np.array(mask_subsc)]

## final ajusts
df_trade.set_index('Date', inplace = True)
df_trade = df_trade.sort_index()


# make a variable global
wallet_dic = {}
global wallet_dic

if not last_pos.empty:
    wallet_dic[date_last_updated_wlt.strftime('%Y%m%d')] = last_pos
    
# financials
cols_fin = ['Date', 'Ticker', 'Profit', '(%)']
df_fin = pd.DataFrame(columns = cols_fin)
global df_fin

df_cash = pd.DataFrame(columns = ['TradeProfit', 'DivJCP'], index = pd.Index([], name = 'Date'))
global df_cash


# Dividendos
df_proventos = df[(df.Evento == 'jcp')
                 | (df.Evento == 'div')]

df_proventos = df_proventos.drop(columns = ['IN_OUT', 'Evento', 'Custodia', 'Quantidade']).set_index('Date')
df_proventos.columns = ['Ticker', 'PU', 'Value']

TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

: 

### DATETIME

In [ ]:
## Datetime 

def get_day_d1(day):
    
    print(f'Dia de data sendo chamado: {day}')
    
    if day.weekday() == 0: # ou seja, d0 for segunda:
        day_d1 = day - dt.timedelta(3) 

    else: 
        day_d1 = day - dt.timedelta(1)

    # still we need to check for hollidays.
    while day_d1 in df_holly.Data.values: # in case that or d1 be a holliday, subtract one day of until it is not anymore.
        day_d1 = day_d1 - dt.timedelta(1)
        
    return day_d1


now = dt.datetime.now().strftime('%Y%m%d-%Hh%Mm%Ss')  #use for export files.


### UPDATING THE WALLET

In [ ]:
# esse dataframe registra qual a posicao que estava em um x dia. Sera snapshots com base na data.
# for starting this dataframe, we need a empty dataframe. For every transaction in df_trade, we need to check 
# if we already had the asset, make the computations and them, at the end of a day, store the new position.
# we can use the yahoo finance library to get the cotations at the end of a day to record together with the pos.


def main(day, df_trade):
    # PARAMETERES
    # day --> string yyyymmdd
    # df_trade_records: dataframe with transactional records, index must be dt.date
    # -------------------------------------------------------------------------------------------------------
    global tst
    
    day_string = day
    day = dt.datetime.strptime(day, '%Y%m%d').date() # to use this as dt.date
    day_d1 = get_day_d1(day)
    print('d-1 calculado: ', day_d1)
    
    net = 0 # net restarts each day.
    
    # 1. --> check transactions of day and update wallet.
    try: 
        df_trade_day = df_trade.loc[day];
        
        
        # this fix a bug of unpack variables when the df has only one transaction.
        if df_trade_day.shape == (5,): 
            df_trade_day = df_trade.loc[[day]]


        # try to get the wallet if exists in the dictionary, otherwise create a empty.
        try: df_wallet = wallet_dic[day_d1.strftime('%Y%m%d')].copy() 
        except KeyError: df_wallet = pd.DateFrame(columns = ['Qt', 'PU', 'ValorAplicado'], 
                                                      index = pd.Index([], name = 'Ticker'))


        # if there is transactions on that day, the wallet will be updated.
        for row in df_trade_day.values:
            

            for DC, Ticker, Qt, PU, ValorAplicado in [row]:


                if DC == 'Credito': # checar na carteira se tem o ativo. Se tiver, soma qtds e altera PM.            


                    if Ticker not in df_wallet.index:

                        new_Qt = Qt
                        new_PU = PU
                        new_ValorAplicado = ValorAplicado            


                    if Ticker in df_wallet.index: # it means there is a new share getting in the wallet for the first time.

                        old_vals = df_wallet.loc[Ticker]

                        new_Qt = old_vals.Qt + Qt
                        new_PU = (old_vals.ValorAplicado + ValorAplicado) / new_Qt
                        new_ValorAplicado = new_PU * new_Qt

    #-------------------------------------------------------------------------------------------------------
                    
                elif DC == 'Debito':

                    old_vals = df_wallet.loc[Ticker]            
                    new_Qt = old_vals.Qt - Qt

                    if new_Qt == 0: # that means, we have 0 shares of this Ticker.
                        new_PU = 0.0
                        new_ValorAplicado = 0.0

                    else:
                        new_PU = old_vals.PU
                        new_ValorAplicado = new_Qt * old_vals.PU

                    
                    # FINANCE;
                    profit = (PU - old_vals.PU) * Qt
                    profit_pcent = (ValorAplicado/(old_vals.PU * Qt) -1) * 100

                    df_fin = pd.concat([df_fin, pd.DateFrame(
                                            [[day, Ticker, profit, profit_pcent]], 
                                                     columns = cols_fin)])

                    net = net + profit


                # NOW UPDATE THE LOCAL WALLET OF THE DAY

                new_record = [new_Qt, new_PU, round(new_ValorAplicado, 2)]
                df_wallet.loc[Ticker] = new_record


            # after all trades records of a day, we can now update our dfs database with the fully updated wallet.

        df_wallet = df_wallet[df_wallet.Qt != 0]     # assim removemos os ativos totalmente liquidados.
        df_wallet['Qt'] = df_wallet.Qt.astype(int) # pq diabos ta saindo float? 
        
        wallet_dic[day_string] = df_wallet
          
        
    except: 
        wallet_dic[day.strftime('%Y%m%d')] = wallet_dic[day_d1.strftime('%Y%m%d')].copy()
        
    #-------------------------------------------------------------------------------------------------------
    ### working with the financials studies.
    #1 TradeProfits:
   
    try: 
        df_cash.loc[day, ['TradeProfit']] = df_cash.loc[day_d1, ['TradeProfit']] + net
    except:
        df_cash.loc[day, ['TradeProfit']] = net
    
    #2 proventos:
    
    try: 
        prov_income = df_proventos.loc[[day], ['Value']].groupby('Date').sum().values
    except: prov_income = 0 # it means that we don't have any income from JCP or div on that day.
    
    try: 
        df_cash.loc[day, ['DivJCP']] = prov_income + df_cash.loc[day_d1, ['DivJCP']].values
    except: 
        df_cash.loc[day, ['DivJCP']] = prov_income
        
    #3 sum of trade profits and income. 
    
 
    return print('Day ', day.strftime('%Y-%m-%d'), 'Pass!')

#### CALL CELL

In [ ]:
# data do ultimo update d carteira + 1
     
try: start = (date_last_updated_wlt + dt.timedelta(1)).strftime('%Y-%m-%d')
except: start = df.Date.min().strftime('%Y-%m-%d')
    
end = (dt.datetime.today() - + dt.timedelta(1)).strftime('%Y-%m-%d')

t = pd.period_range(start=start,
                  end=end,
                  freq='D')

for day in t:
    
    #if day.weekday != 6 and day.weekday != 5  and day.to_timestamp() not in df_holly.Data.values: # pula sabado/domingos e
    main(day.strftime('%Y%m%d'), df_trade) ## REAL CALL


### BUT as we used the last day already record on or dataset, we need to delet that very first day from our dictionary, for not duplicate data;

if date_last_updated_wlt:
    wallet_dic.pop(date_last_updated_wlt.strftime('%Y%m%d'))

Dia de data sendo chamado: 2023-09-01
d-1 calculado:  2023-08-31
Day  2023-09-01 Pass!
Dia de data sendo chamado: 2023-09-02
d-1 calculado:  2023-09-01
Day  2023-09-02 Pass!
Dia de data sendo chamado: 2023-09-03
d-1 calculado:  2023-09-02
Day  2023-09-03 Pass!
Dia de data sendo chamado: 2023-09-04
d-1 calculado:  2023-09-01
Day  2023-09-04 Pass!
Dia de data sendo chamado: 2023-09-05
d-1 calculado:  2023-09-04
Day  2023-09-05 Pass!
Dia de data sendo chamado: 2023-09-06
d-1 calculado:  2023-09-05
Day  2023-09-06 Pass!
Dia de data sendo chamado: 2023-09-07
d-1 calculado:  2023-09-06
Day  2023-09-07 Pass!
Dia de data sendo chamado: 2023-09-08
d-1 calculado:  2023-09-07
Day  2023-09-08 Pass!
Dia de data sendo chamado: 2023-09-09
d-1 calculado:  2023-09-08
Day  2023-09-09 Pass!
Dia de data sendo chamado: 2023-09-10
d-1 calculado:  2023-09-09
Day  2023-09-10 Pass!
Dia de data sendo chamado: 2023-09-11
d-1 calculado:  2023-09-08
Day  2023-09-11 Pass!
Dia de data sendo chamado: 2023-09-12
d-1 c

In [ ]:
### Some modifications after, cuz it was more easily done and readable

df_cash['TotalEarnings'] = df_cash.TradeProfit + df_cash.DivJCP
df_cash.head(2)

,TradeProfit,DivJCP,TotalEarnings
Date,,,
2023-09-01,0,97.84,97.84
2023-09-02,0,97.84,97.84


In [ ]:
### before the slow get price, we can work on the dataframe that records the profit made by SELLING stocks;

df_cash

,TradeProfit,DivJCP,TotalEarnings
Date,,,
2023-09-01,0,97.84,97.84
2023-09-02,0,97.84,97.84
2023-09-03,0,97.84,97.84
2023-09-04,0,97.84,97.84
2023-09-05,0,97.84,97.84
2023-09-06,0,97.84,97.84
2023-09-07,0,97.84,97.84
2023-09-08,0,98.12,98.12
2023-09-09,0,98.12,98.12


In [ ]:
### ADD COTAÇÃO NA CARTEIRAS ATRAVÉS DO DICIONÁRIO; - FIZ SEPARADO PARA NÃO TER QUE MODIFICAR O MAIN;

wallet_wprices = {}
# GET MARKET PRICES

def get_MarketPrice(dia, df):
    
    dia = dt.datetime.strptime(dia, '%Y%m%d')
    diaMais  = dia + dt.timedelta(5)
    
    #date as string;
    dia_str = dia.strftime('%Y-%m-%d')
    diaMais_str = diaMais.strftime('%Y-%m-%d')
    
    #---
    tickers = df.index
    tickers = [x+'.SA' for x in tickers]

    df_prices = yf.download(tickers, start = dia_str, end = diaMais_str).iloc[[0]].Close.T
    df_prices.applymap(lambda x: round(x, 2))
    df_prices.columns = ['PUMercado']
    df_prices.index = [x[:-3] for x in df_prices.index] 
    
    #join and calc:   
    
    df = df.join(df_prices)
    df['ValorMercado'] = round(df.PUMercado * df.Qt, 2)
    df['Return (%)'] = round((df.PUMercado/ df.PU)*100 - 100, 2)
    
    #solving NAN PROBLEM with ENBR;
    df = df.fillna(0)
    
    return df


# chamada da função.
for dia, dataframe in wallet_dic.items():
    
    df = get_MarketPrice(dia, dataframe)
    wallet_wprices[dia] = df

    

[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*****

### WHAT ABOUT OR DATA BASE FOR THE CASH ENVOLVED WITHIN THE OPERATIONS?

In [ ]:
df_cash.head()

,TradeProfit,DivJCP,TotalEarnings
Date,,,
2023-09-01,0,97.84,97.84
2023-09-02,0,97.84,97.84
2023-09-03,0,97.84,97.84
2023-09-04,0,97.84,97.84
2023-09-05,0,97.84,97.84


In [ ]:
### CASH SQL FEEDER FILE TXT WRITTING

sentc_cash = 'INSERT INTO cash (data, valor_trade, valor_earnings, total_profit) values\n'
rows = []
for index, vals in zip(df_cash.index, df_cash.values):
    c1, c2, c3 = vals 
    row = '("{}", {}, {}, {})'.format(index.strftime('%Y%m%d'), round(c1, 2), round(c2, 2), round(c3, 2))
    rows.append(row)
    
sentc_cash = sentc_cash + ",\n".join(rows)

In [ ]:
arq = '/home/peon/Downloads/cash_update_' + now + '.txt'

with open(arq, 'w') as f:

    f.write(sentc_cash)


In [ ]:
# %%sql

# create table cash (
#     id int not null auto_increment,
#     data date default null,
#     valor_trade decimal(14,2) default null,
#     valor_earnings decimal(14,2) default null,
#     total_profit decimal(14,2) default null,
#     primary key (id)
#     );

### FEED SQL PART I

#### 1. Arranging the data of the wallet dicionary

In [ ]:
## bloco prepara os dados da carteira diária e com cotação para inputar em base SQL;

df_carteira_cpt = pd.DataFrame([],
                               columns = ['Ticker', 'Qt', 'PU', 'ValorAplicado', 'PUMercado', 'ValorMercado', 'Return (%)'],
                              index = pd.Index([], name = 'Date')
                              )

                               
for dia, carteira in wallet_wprices.items():
    
    diaf = dia[:4] + '-' + dia[4:6] + '-' + dia[6:]
    carteira = carteira.reset_index()
    carteira.index = pd.Index(carteira.shape[0] * [diaf], name = 'Date')
    
    df_carteira_cpt = pd.concat([df_carteira_cpt, carteira])
    
    
df_carteira_cpt = df_carteira_cpt.reset_index()    

#### 2. SQL Create Table;

In [ ]:
# %%sql  
# DROP TABLE Carteira;
# CREATE TABLE Carteira 
# (
# Date DATE, 
# Ticker VARCHAR(64), 
# Qt INT, 
# PU DECIMAL(6,2),
# ValorAplicado DECIMAL(8,2), 
# PUMercado DECIMAL(6,2),
# ValorMercado DECIMAL(8,2),
# `Return` DECIMAL(6,2)
# );


#### 3. Transf data into SQL INPUT STATEMENT;

In [ ]:
stc_list = []
revalues = []

def write_sql_input_statement(df, sql_table):

    df[['Date', 'Ticker']] = df[['Date', 'Ticker']].apply(lambda x: '"' + x + '"')
    
    for row in df.values:

        stc = '(' + ", ".join(row.astype(str)) + ')'
        stc_list.append(stc)

    # after creating the array:
    stc_final = ',\n'.join(stc_list)
    
    S = "USE my_wallet;\nINSERT INTO \n{0} \nVALUES \n{1}".format(sql_table, stc_final)
    
    arq = '/home/peon/Downloads/carteira_update_' + now + '.txt'

    with open(arq, 'w') as f:

        f.write(S)
    
    return print("SQL File Written!")

write_sql_input_statement(df_carteira_cpt, 'Carteira')

run_end = dt.datetime.now()
run_end

SQL File Written!


datetime.datetime(2024, 1, 19, 2, 11, 52, 697490)

In [ ]:
# TIME RUNNING:

delta_run = run_end - run_start
delta_run

datetime.timedelta(seconds=482, microseconds=524748)

In [ ]:
delta_run.seconds/60

8.033333333333333

In [ ]:
df_trade[df_trade.Ticker == 'BBSE3']

,IN_OUT,Ticker,Quantidade,PU,ValorAplicado
Date,,,,,
2023-09-27,Debito,BBSE3,65,32.05,2083.25
2023-10-18,Debito,BBSE3,2,31.72,63.44
2023-11-03,Credito,BBSE3,20,30.72,614.40
2023-11-16,Debito,BBSE3,108,31.50,3401.89


In [ ]:
df